In [ ]:
import pandas as pd
import numpy as np

# ============================================================================
# CARGA DE DATOS
# ============================================================================

# Cargar TopRepositorios
df_top_repos = pd.read_csv('../Datos/TopRepositorios.csv')

# Cargar TopReposXLenguajes.txt
df_repos_lang = pd.read_csv('../Datos/TopReposXLenguajes.csv')

In [ ]:
# ============================================================================
# LIMPIEZA DE DATOS
# ============================================================================

def limpiar_numeros(df, columnas):
    """
    Convierte columnas con números formateados (e.g., '6,702') a enteros
    """
    for col in columnas:
        df[col] = df[col].str.replace(',', '').str.replace('"', '').astype(int)
    return df

# Limpiar números en TopRepositorios
df_top_repos = limpiar_numeros(df_top_repos, ['NumberOfStar', 'NumberOfFork'])

# Limpiar números en TopReposXLenguajes
df_repos_lang = limpiar_numeros(df_repos_lang, ['NumberOfStar', 'NumberOfFork'])

In [ ]:
# ============================================================================
# ESTADÍSTICAS GENERALES
# ============================================================================

print(f"- Total de lenguajes únicos: {df_repos_lang['Language'].nunique()}")
print(f"- Repositorio más popular: {df_top_repos.loc[df_top_repos['NumberOfStar'].idxmax(), 'Repository']}")
print(f"- Total de estrellas (top repos): {df_top_repos['NumberOfStar'].sum():,}")
print(f"- Total de forks (top repos): {df_top_repos['NumberOfFork'].sum():,}")

In [ ]:
# ============================================================================
# ANÁLISIS POR LENGUAJE
# ============================================================================

# Estadísticas agregadas por lenguaje
stats_por_lenguaje = df_repos_lang.groupby('Language').agg({
    'NumberOfStar': ['sum', 'mean', 'count'],
    'NumberOfFork': ['sum', 'mean']
}).round(0)

stats_por_lenguaje.columns = ['Total_Stars', 'Promedio_Stars', 'Num_Repos', 'Total_Forks', 'Promedio_Forks']
stats_por_lenguaje = stats_por_lenguaje.reset_index()
stats_por_lenguaje = stats_por_lenguaje.sort_values('Total_Stars', ascending=False)

print("\nTop 10 Lenguajes por Total de Estrellas:")
print(stats_por_lenguaje.head(10))

In [ ]:
# ============================================================================
# PREPARAR DATOS PARA VISUALIZACIÓN
# ============================================================================

# 1. Promedio de estrellas por lenguaje (para gráfico de barras)
promedio_estrellas = stats_por_lenguaje[['Language', 'Promedio_Stars', 'Num_Repos']].copy()
promedio_estrellas = promedio_estrellas.sort_values('Promedio_Stars', ascending=False).head(10)

# 2. Top repositorios por lenguaje específico (para análisis detallado)
top_repos_por_lang = {}
for lang in df_repos_lang['Language'].unique():
    repos = df_repos_lang[df_repos_lang['Language'] == lang].nlargest(5, 'NumberOfStar')
    top_repos_por_lang[lang] = repos

# 3. Distribución de repos por lenguaje
distribucion_lenguajes = df_repos_lang['Language'].value_counts().reset_index()
distribucion_lenguajes.columns = ['Language', 'Count']

In [ ]:
# ============================================================================
# EXPORTAR DATOS PROCESADOS
# ============================================================================

# Guardar archivos procesados
df_top_repos.to_csv('../Datos_procesados/Top_repos_clean.csv', index=False)
df_repos_lang.to_csv('../Datos_procesados/Repos_por_lenguaje_clean.csv', index=False)
stats_por_lenguaje.to_csv('../Datos_procesados/Estadisticas_lenguajes.csv', index=False)
promedio_estrellas.to_csv('../Datos_procesados/Promedio_estrellas_top10.csv', index=False)
distribucion_lenguajes.to_csv('../Datos_procesados/Distribucion_lenguajes.csv', index=False)